In [1]:
from spoofnet_model import SpoofNet
import torch

In [3]:
checkpoint_path = "mobilenet_training_attempt2/mobilenetv2-best.pt"
model = SpoofNet()
checkpoint = torch.load(checkpoint_path, map_location="cpu")
model.load_state_dict(checkpoint['state_dict'])
model.eval()

SpoofNet(
  (pretrained_net): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_siz

In [4]:
scripted_model = torch.jit.script(model)
scripted_model.save("mobilenet_training_attempt2/mobilenetv2-scripted.pt")
#tidak ada quantize dan pruning 
print("✅ Model berhasil dikonversi ke TorchScript!")

✅ Model berhasil dikonversi ke TorchScript!


In [10]:
import os
input_dir ="Dataset_spoofing"
train_dir = os.path.join(input_dir, 'LCC_FASD_training')
val_dir = os.path.join(input_dir, 'LCC_FASD_development')
test_dir = os.path.join(input_dir, 'LCC_FASD_evaluation')

In [11]:
dataset_dir = [dir for dir in sorted(os.listdir(input_dir)) if os.path.isdir(os.path.join(input_dir, dir))]
label_name = [subdir for subdir in sorted(os.listdir(train_dir)) if os.path.isdir(os.path.join(train_dir, subdir))]

In [12]:
print(f"Main directories\t: {input_dir}")
print(f"Dataset sub-directories\t: {dataset_dir}")
print(f"Train set directory\t: {label_name}")

Main directories	: Dataset_spoofing
Dataset sub-directories	: ['LCC_FASD_development', 'LCC_FASD_evaluation', 'LCC_FASD_training']
Train set directory	: ['real', 'spoof']


In [13]:
import os
import glob
import numpy as np
import pandas as pd

dir_dict = {'train': train_dir, 'val': val_dir, 'test': test_dir}
case_count, img_disp, set_length  = {}, {}, {}

for key, val in dir_dict.items():
    case_count[key] = {}
    img_disp[key] = {}
    set_count = 0
    
    for label in label_name:
        label_list = list(sorted(glob.glob(os.path.join(val, label, "*.png"))))
        if len(label_list) == 0:
            continue

        case_count[key][label] = len(label_list)
        set_count += len(label_list)
        
        select_img_id = np.random.randint(len(label_list)-1)
        img_disp[key][label] = label_list[select_img_id]
        
    set_length[key] = set_count

# Konversi ke DataFrame
case_count_df = pd.DataFrame(case_count)

# Menghitung persentase
percentage_df = case_count_df.div(case_count_df.sum(axis=0), axis=1) * 100

# Gabungkan dengan DataFrame awal
dataset_summary = case_count_df.copy()
dataset_summary["Total"] = case_count_df.sum(axis=1)
dataset_summary["Train (%)"] = percentage_df["train"]
dataset_summary["Val (%)"] = percentage_df["val"]
dataset_summary["Test (%)"] = percentage_df["test"]

print(f"Dataset summary:\n\n{dataset_summary}")


Dataset summary:

       train   val  test  Total  Train (%)    Val (%)  Test (%)
real    1223   405   314   1942  14.736715  13.738128   4.14248
spoof   7076  2543  7266  16885  85.263285  86.261872  95.85752


In [14]:
# Visualizing some of the data set
from skimage.io import imread

num_classes = len(label_name)
num_dataset = 0
for key, val in set_length.items():
  num_dataset += 1 if val > 0 else 0

f, ax = plt.subplots(num_classes, num_dataset, figsize=(num_dataset*10, 18))

for k in range(num_classes*num_dataset):
    j, i = k//num_dataset, k%num_dataset  # Image indexing
    img_disp_df = pd.DataFrame(img_disp) 
    img = imread(img_disp_df.iloc[j, i])
    ax[j, i].imshow(img, cmap='gray')
    ax[j, i].set_title(f"{img_disp_df.columns[i].upper()}: {img_disp_df.index[j].capitalize()}", fontsize=32)
    ax[j, i].axis('off')
    ax[j, i].set_aspect('auto')
plt.show()

NameError: name 'plt' is not defined

In [16]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import warnings
warnings.filterwarnings('ignore')
# Define the data transformations for training, validation, and testing
train_transform = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.augmentations.transforms.ISONoise(
        color_shift=(0.15, 0.35), 
        intensity=(0.1, 0.5), 
        always_apply=False, 
        p=0.05),
    A.augmentations.transforms.RandomBrightnessContrast(
        brightness_limit=(-0.2, 0.2),
        contrast_limit=(-0.2, 0.2),
        always_apply=False,
        brightness_by_max=True, p=0.125),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.augmentations.transforms.ImageCompression(
        quality_lower=50,
        quality_upper=100,
        always_apply=False,
        p=0.25),
    A.augmentations.dropout.CoarseDropout(
        max_holes=24,
        max_height=8,
        max_width=8,
        min_holes=4,
        min_height=4,
        min_width=4,
        fill_value=0,
        always_apply=False,
        p=0.25),
    A.augmentations.transforms.GaussNoise(
        var_limit=(10.0, 50.0),
        mean=0,
        always_apply=False,
        p=0.2),
    A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=0, shift_limit=0, p=1.0),
    A.augmentations.geometric.transforms.ShiftScaleRotate(
        shift_limit=0.1, 
        scale_limit=0.1, 
        rotate_limit=15, 
        interpolation=cv2.INTER_CUBIC, 
        border_mode=cv2.BORDER_REFLECT_101, 
        always_apply=False, 
        p=0.5),
    A.augmentations.geometric.transforms.Affine(
        scale=(0.5, 1.5), 
        translate_percent=(0.1, 0.2), 
        rotate=(-15, 15), 
        shear=(-8, 8), 
        interpolation=cv2.INTER_CUBIC, 
        always_apply=False, 
        p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])



E:\2025\pythonProject\.venv\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [17]:
val_transform = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    # To tensor
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    # To tensor
    ToTensorV2(),
])

In [18]:
def gen_df(img_dir):
    real_img_dir = os.path.join(img_dir, 'real')
    spoof_img_dir = os.path.join(img_dir, 'spoof')
    real_files = [os.path.join(real_img_dir, f) for f in os.listdir(real_img_dir) if os.path.isfile(os.path.join(real_img_dir, f))]
    spoof_files = [os.path.join(spoof_img_dir, f) for f in os.listdir(spoof_img_dir) if os.path.isfile(os.path.join(spoof_img_dir, f))]

    # Create DataFrame
    data = {
        'file_path': real_files + spoof_files,
        'live': [1] * len(real_files) + [0] * len(spoof_files)
    }
    
    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

In [19]:
df_train = gen_df(dir_dict['train'])
df_val = gen_df(dir_dict['val'])
df_test = gen_df(dir_dict['test'])

In [20]:
df_train['live'].value_counts()

live
0    7076
1    1223
Name: count, dtype: int64

In [21]:
df_train['live'][0]

0

In [22]:
df_train_0 = df_train[df_train['live']==0][:1223]
df_train_1 = df_train[df_train['live']==1][:1223]
df_train_balanced = pd.concat([df_train_0, df_train_1]).reset_index(drop=True)
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_train_balanced

,file_path,live
0,Dataset_spoofing\LCC_FASD_training\real\IPHONE...,1
1,Dataset_spoofing\LCC_FASD_training\real\YOUTUB...,1
2,Dataset_spoofing\LCC_FASD_training\spoof\YOUTU...,0
3,Dataset_spoofing\LCC_FASD_training\real\YOUTUB...,1
4,Dataset_spoofing\LCC_FASD_training\spoof\YOUTU...,0
...,...,...
2441,Dataset_spoofing\LCC_FASD_training\real\YOUTUB...,1
2442,Dataset_spoofing\LCC_FASD_training\spoof\YOUTU...,0
2443,Dataset_spoofing\LCC_FASD_training\spoof\FT720...,0
2444,Dataset_spoofing\LCC_FASD_training\real\YOUTUB...,1


In [23]:
df_train_balanced['live'].value_counts()

live
1    1223
0    1223
Name: count, dtype: int64

In [24]:
df_val['live'].value_counts()

live
0    2543
1     405
Name: count, dtype: int64

In [25]:
df_val_0 = df_val[df_val['live']==0][:405]
df_val_1 = df_val[df_val['live']==1][:405]
df_val_balanced = pd.concat([df_val_0, df_val_1]).reset_index(drop=True)
df_val_balanced = df_val_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_val_balanced['live'].value_counts()

live
0    405
1    405
Name: count, dtype: int64

In [26]:
df_test['live'].value_counts()

live
0    7266
1     314
Name: count, dtype: int64

In [27]:
df_test_0 = df_test[df_test['live']==0][:314]
df_test_1 = df_test[df_test['live']==1][:314]
df_test_balanced = pd.concat([df_test_0, df_test_1]).reset_index(drop=True)
df_test_balanced = df_test_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_test_balanced['live'].value_counts()

live
1    314
0    314
Name: count, dtype: int64

In [29]:
# pytorch
import torch
import torch.nn as nn

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import DataLoader, Dataset
class FASDataset(Dataset):
    
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['file_path']
        label = self.df.iloc[idx]['live']
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        image = Image.open(img_path)
        image = np.array(image)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        return image, label

In [30]:
train_dataset = FASDataset(df_train, train_transform)
val_dataset = FASDataset(df_val, val_transform)
test_dataset = FASDataset(df_test, test_transform)

dataloader_train = DataLoader(train_dataset, batch_size=32, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=32, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [31]:
for inputs, labels in dataloader_val:
    print(inputs.shape, inputs.dtype)
    break

torch.Size([32, 3, 224, 224]) torch.float32
